## Yarkovsky Effect - Maximized

This example demonstrates how to add a simplified and maximized version of the Yarkovsky effect (Veras, Higuchi, and Ida, 2019) to a Rebound simulation. This version requires fewer parameters, but is not as accurate as the full version. It places an upper bound on how far this effect will push an object inwards or outwards.

First we setup a Rebound simulation with the Sun and a test particle (which will be our asteroid) at .5 AU

In [11]:
import rebound
import reboundx
import numpy as np

#Simulation begins here
sim = rebound.Simulation()

sp = sim.particles #simplifies way to access particles parameters 

sim.units = ('yr', 'AU', 'Msun') #changes simulation and G to units of solar masses, years, and AU  
sim.integrator = "whfast" #integrator for sim
sim.dt = .05 #timestep for sim

sim.add(m=1) #Adds Sun 
sim.add(a=.5, f=0, Omega=0, omega=0, e=0, inc=0, m=0) #Adds test particle 

#Moves all particles to center of momentum frame
sim.move_to_com()

#Gives orbital information before the simulation begins
print("\n***INITIAL ORBITS:***")
for orbit in sim.calculate_orbits():
    print(orbit)


***INITIAL ORBITS:***
<rebound.Orbit instance, a=0.5000000000000001 e=1.799893761170345e-16 inc=0.0 Omega=0.0 omega=0.0 f=0.0>


We then add the maximized Yarkovsky effect from Reboundx, and adds the required parameters. The effect requires you to add a luminosity to the simulation and a radius and density for each object you want the force to act on. The parameters must be in mks units. We also must determine if we want our particle to drift inwards or outwards using the direction flag. Setting it to 1 will push the object outwards while setting it to -1 will push the object inwards. We'll choose outwards for this example.

In [12]:
rebx = reboundx.Extras(sim)
yark = rebx.load_force("max_yarkovsky")

yark.params["lstar"] = 3.828e26
sp[1].params["body_density"] = 3000
sp[1].params["direction_flag"] = 1
sp[1].r = 1000 

rebx.add_force(yark)

We integrate this system for 50,000 years and print out the difference between the particle's semi-major axis before and after the simulation.

In [13]:
tmax=50000 # in yrs

a_start = .5 #starting semi-major axis for the asteroid
            
sim.integrate(tmax) #integrates system for tmax years       
 
a_final = sp[1].a #semi-major axis of asteroid after the sim    
                      
print("CHANGE IN SEMI-MAJOR AXIS", a_final-a_start, "AU") #prints difference between the intitial and final semi-major axes of asteroid


CHANGE IN SEMI-MAJOR AXIS 2.123604262072032e-05 AU
